In [1]:
#Paqueterias necesarias para la realizacion de todo el códigatzo
import numpy as np
import math
from copy import copy, deepcopy
import random
import statistics

#Solo necesitas meter el txt en la celda de aquí abajo. 
(De favor, el txt debe estar como en el ejemplo de entrada en el manual de metaheuristica). Sino, te tocará meter las entradas a manita xd

In [2]:
bt = "/content/drive/MyDrive/Computo evolutivo/input.txt"

#Poner el número de ejecuciones que deseas. En mi caso, le puse 6

In [3]:
M = 6

#Lector del txt, y organizador las entradas para poder ser compiladas en las funciones

In [4]:
def instancia_viajero(texto):
  with open(texto) as f: #Abro el txt
    contents = f.readlines() #Leo el contenido del mismo

  cities = contents[0][:-1] #Agarro todo el texto de la primera fila, excepto por el salto de línea
  cities = int(cities) #Tranformo de string a entero el número

  iterations = contents[1][:-1] #Hago lo mismo que con cities, pero ahora en la segunda fila
  iterations = int(iterations)

  costs = [] #Creo una lista vacía, donde voy a ir metiendo los costos
  ahsu = '' #Esta es la herramienta mágica :v
  

  for i in range(2,len(contents)): #Como las primeras dos líneas corresponden a entradas diferentes, empiezo en dos. Porque el contenido de aquí para abajo es de la entrada 3
    if i < len(contents)-1: #Si todavía no llego a la última fila
      for j in range(len(contents[i])): #Recorre la fila i
        if contents[i][j] != ' ' and contents[i][j] != '\n': #Si es diferente a un espacio o un salto de línea
          ahsu += contents[i][j] #Añade ese texto a la herramienta mágica.
        else:
          ahsu = int(ahsu) #Convierte el texto a entero
          costs.append(ahsu) #Metelo en la lista de costos
          ahsu = '' #Vacía la herramienta mágica
    else: #Si ya estás en la última fila.
      ahsu = contents[i]  #La herramienta mágica toma todo el texto (número)
      ahsu = int(ahsu) #Conviertelo a entero
      costs.append(ahsu) #Agregalo a la lista de costos
      ahsu = '' #Vacíalo y a mimir
  
  return cities, iterations, costs #Regresa las entradas(1, 2, 3)


ins = instancia_viajero(bt)

In [5]:
#Ciudades, entrada 1
ins[0]

10

In [6]:
#Iteraciones, entrada 2
ins[1]

100

In [7]:
#Costos (se ve medio gacho en vertical, pero pues sirve xd). Por eso mejor lo comento, pero si quieres ver la lista de los pesos, solo quitale el hashtag (comentario)
#Entrada 3

#ins[2]

#Matriz simetrica con los costos de ir de una ciudad a otra

In [8]:
def make_sym_matrix(n,vals): #n indica el número de ciudades, y vals la lista de los costos
  m = np.zeros([n,n], dtype=np.double) # Genera una matriz de ceros de nxn, en este caso n = 10.
  xs,ys = np.triu_indices(n,k=1)

  #Las posiciones [i,r] y [r,i] tienen el mismo valor, porque busco generar una matriz simetrica
  m[xs,ys] = vals 
  m[ys,xs] = vals

  m[ np.diag_indices(n) ] = float('inf') #En la diagonal les doy un valor infinito, para así evitar elegir esas casillas. Porque, por ejemplo, no puede ir de la ciudad 0 a ella misma
  return m #Regreso la matriz generada

In [9]:
#Esta es la lista de los costos de ir de A ciudad a B ciudad, por eso atob... a to b :vvv
#De haber leído la parte de los 20 puntos antes, me habría ahorrado esto :):

#atob = [49, 30, 53, 72, 19, 76, 87, 45, 48, 19, 38, 32, 31, 75, 69, 61, 25, 41, 98, 56, 6, 6, 45, 53, 52, 29, 46, 90, 23, 98, 63, 90, 69, 50, 82, 60, 88, 41, 95, 61, 92, 10, 82, 73, 5]

In [10]:
#Aquí ya se puede ver como queda la matriz simetrica. Donde lo que está en el triangulo superior es lo mismo que en el triangulo inferior.
kk = make_sym_matrix(ins[0], ins[2]); kk

array([[inf, 49., 30., 53., 72., 19., 76., 87., 45., 48.],
       [49., inf, 19., 38., 32., 31., 75., 69., 61., 25.],
       [30., 19., inf, 41., 98., 56.,  6.,  6., 45., 53.],
       [53., 38., 41., inf, 52., 29., 46., 90., 23., 98.],
       [72., 32., 98., 52., inf, 63., 90., 69., 50., 82.],
       [19., 31., 56., 29., 63., inf, 60., 88., 41., 95.],
       [76., 75.,  6., 46., 90., 60., inf, 61., 92., 10.],
       [87., 69.,  6., 90., 69., 88., 61., inf, 82., 73.],
       [45., 61., 45., 23., 50., 41., 92., 82., inf,  5.],
       [48., 25., 53., 98., 82., 95., 10., 73.,  5., inf]])

#Generar solución inicial, usando greedy

In [11]:
def sol_inicial(redi):
  ini = [] #Lista vacía, donde voy a guardar los costos del movimiento de una ciudad a otra
  per = [] #Lista vacía donde voy a guardar el indice de esa ciudad
  kha = deepcopy(redi) #Hago un deep copy de la matriz simetrica, por si la riego (Sí pasó, varias veces xd)
  x = 0 #Esta es la herramienta mágica

  for i in range(len(kha)): #Creo un for, donde voy a correrlo n veces, o sea, popr la cantidad de ciudades.
    per.append(x) #Aquí meto de una la ciudad cero, porque siempre parto de ahí
    if i != len(kha)-1: #Al final, la última ciudad no tiene de otra más que ir a cero, por eso para cuando eso pasa cree un else.
      ini.append(kha[x].min()) #Agrego el valor mínimo de ir a la ciudad x a cualquier ciudad disponible a la lista.
      #print(ini)
      #print(x)
      for j in range(len(kha)): #Toda la columna correspondiente a la ciudad x se vuelve infinito, porque no quiero que elijan esa ciudad, debido a que ya pasé por ahí
        kha[j,x] = float('inf') # Le doy un valor de infinito
        #print(kha)
      x = np.where(kha[x]==kha[x].min()) #Ahora, agarro el indice de la ciudad. Por ejemplo, de la ciudad 0 va a 5, entonces ahora x = 5.
      x = x[0][0] #Y con esto agarra únicamente el número, y no lo agarro como array. Esto me solucionó problemas que tenía.
    else:
      ini.append(kha[0].min()) #Si ya estoy en la última ciudad, simplemente voy a la fila 0, y le pongo el costo de ir de esa ciudad a 0.
      #print(ini)
  return per, sum(ini) #Regreso la permutación y la suma del costo de la permutación

per_costo_inicial = sol_inicial(kk); per_costo_inicial

([0, 5, 3, 8, 9, 6, 2, 7, 1, 4], 271.0)

#Generar vecindarios, lista tabu, iteraciones y regresar la permutación de menor costo, con el valor del costo

In [12]:
permutacion = per_costo_inicial[0]; permutacion

[0, 5, 3, 8, 9, 6, 2, 7, 1, 4]

In [13]:
costo = per_costo_inicial[1]; costo

271.0

In [14]:
def iteraciones(n, ite, p, c): #n = número de ciudades, ite = número de iteraciones, p = permutación greedy, c = costo de la permutación greedy

  tabu = [] #Lista tabu, empieza estando vacía, por obvias razones jaja
  mejor_p = p #La mejor permutación empieza siendo la generada por greedy
  mejor_c = c #Y aquí el costo de dicha permutación

  while ite > 0:
    #print(p)
    dictOfPer = { i : p[i] for i in range(0, len(p) ) } #Genero un diccionario donde cada ciudad tiene su indice en la permutación original
    #print(dictOfPer)
    copia_dictOfPer = dictOfPer.copy() #Le hago una copia

    if len(copia_dictOfPer) == n: #Ya con la copia hecha, elimino la ciudad 0, porque esa no se mueve.
      del  copia_dictOfPer[0]
      azar = random.choice(list(copia_dictOfPer.values())) #Elijo la ciudad random.
    else: #Y si ya está eliminada, pues solo sigo eligiendo la ciudad random a cambiar de posición en la iteración
      azar = random.choice(list(copia_dictOfPer.values()))
    #print(azar)
    

    for i in range(len(dictOfPer)): #Va a correr esto n veces
      if dictOfPer[i] == azar: #Si el valor del diccionario en la posición i es igualigual a azar(ciudad random elegida), entonces entro.
        tabu.append([i,azar]) #Aquí inserto a la lista tabu la llave y la ciudad.
        tabu.append(n+2//2) #Le pongo n+2, porque sin siquiera hacer una permutación nueva ya le quito 1 al tiempo tabu.
                            #Inserto el tiempo tabu, seguido de la lista con [llave, ciudad]


    #Vecindarios
    vecindario = []
    for i in range(len(p)): #Voy a correr n veces esto.
      if i != 0 and i != tabu[-2][0]: #Aquí debe ser diferente de i = 0 e i = ciudad random. Porque no quiero generar permutación donde la ciudad random se cambie con la ciudad 0
        popis = dictOfPer.copy()                                                             #y tampoco donde la ciudad random se cambie con ella misma, sería la misma permutación
        popis[i], popis[tabu[-2][0]] = popis[tabu[-2][0]], popis[i] #Cambio la ciudad random con la ciudad en la llave i.
        vecindario.append(list(popis.values())) #Hago append al vecindario con el diccionario en forma de lista, para solo tener en vecindario las n-2 permutaciones


    #Costos de las permutaciones del vecindario
    vec_costos = []
    for i in range(len(vecindario)): #Va a correr esto n-2 veces
      suma = 0
      for j in range(len(vecindario[i])): #Ahora lo correrá n veces
        if j < len(vecindario[i])-1: #Si todavía no llego a la última ciudad, corre esto
          suma += kk[vecindario[i][j]][vecindario[i][j+1]] #Entra a la matriz, y sumale el costo de ir de la ciudad j a la ciudad j+1
        else:
          suma += kk[0][vecindario[i][j]] #Si ya estoy en la última ciudad, sumale el costo de ir de la ciudad j a la ciudad 0
      vec_costos.append(suma) #Mete en la lista la suma del costo de la permutación en vecindario[i]


    #Descartar permutaciones que no son posibles, por tener ciudades en la lista tabu que no pueden estar en ciertas zonas de la permutación
    for v in range(len(vec_costos)): #Corro esto n-2 veces
      for tb in range(0,len(tabu),2): #Como tengo la lista con [llave, ciudad] seguida de su tiempo tabu, entonces quiero solo agarrar lo que se encuentra en [llave, ciudad]
        nueva = { i : vec_costos[i] for i in range(0, len(vec_costos) ) } #Genero un diccionario de los costos.                                     #por eso tb da salto de 2.
        if tabu[tb][1] == vecindario[min(nueva, key=nueva.get)][tabu[tb][0]]: #Si tabu[ciudad] es igualigual a vecindario[permutacion de costo minimo][llave]
          vec_costos[min(nueva, key=nueva.get)] = float('inf')  #Poner infinito como coste de esa permutación. Porque no está disponible para elegir,
                                                                                                              #debido a que una ciudad en esa posición se encuentra en la lista tabu

    #Vecindario con coste minimo
    menor_costo = min(nueva, key=nueva.get) #Agarro la llave del menor costo

    #Nueva permutación
    p = vecindario[menor_costo] #Elijo la permutación que corresponde al menor costo

    #Costo de la permutacion
    m = min(vec_costos) #Elijo el menor costo, el cual corresponde a la permutación elegida

    #Mejor permutacion hasta ahora
    if mejor_c > m: #Si el costo de la permutación nueva es menor a la mejor hasta ahora
      mejor_c = m #El menor coste hasta ahora es el nuevo costo.
      mejor_p = p #La mejor permutación hasta ahora, es la nueva permutación

    #Tiempo tabu
    if len(tabu) > 0: #Si tabu no es vacía
      for t in range(1,len(tabu),2): #Recorrer la lista tabu, empezando en 1 (Porque en las posiciones impar se encuentra su tiempo tabu) y dando saltos de 2 por lo mencionado en los parentesis
        tabu[t] = tabu[t] - 1 #Restarle 1 a su tiempo tabu
    if tabu[1] == 0: #Como no pueden entrar dos tiempos tabu al mismo tiempo, siempre el que está adelante es el de menor tiempo, y por ende si su tiempo = 0
      tabu.remove(tabu[1]) #Eliminar el tiempo tabu
      tabu.remove(tabu[0]) #Eliminar  la llave y su ciudad


    ite -= 1 #Como esto cuenta como una iteración, le resto 1 cada que termina
    #print(p)
    #print(m)
    #print(tabu)
  return mejor_p, mejor_c #Regreso la mejor permutación y su costo en las 'ite' iteraciones

#Si quieres modificar el número de iteraciones que vienen en el txt, cambia ins[1] por las iteraciones deseadas
#iteraciones(ins[0],ins[1],permutacion,costo)

#Realización de las M ejecuciones

In [37]:
#Mejor solución encontrada en las M ejecuciones
def mejor_sol_M(M):
  bests = [iteraciones(ins[0],ins[1],permutacion,costo) for _ in range(M)] #Correr M veces la función donde se obtiene la mejor permutación y su costo despues de i iteraciones
  

  #Mejor y 'pior' solución
  best = [] #Lista donde voy a guardar la mejor solución
  worst = [] #Lista donde voy a guardar la peor solución
  costes = [] #Lista donde voy a guardar los costos
  for b in range(len(bests)): #Correr esto M veces
    costes.append(bests[b][1]) #Meter en la lista de costes, el costo de la permutación en la posición b
    #print(costes)
  diccio = { i : costes[i] for i in range(0, len(costes) ) } #Diccionario de los costes
  #print(diccio)
  mejor_permu = min(diccio, key=diccio.get) #Obtener la llave de la permutación con menor costo
  pior_permu = max(diccio, key=diccio.get) #Obtener la llave de la permutación con mayor costo
  best.append(bests[mejor_permu]) #Agregar la mejor permutación con su costo
  worst.append(bests[pior_permu]) #Agregar la pior permutación con su costo


  #Mediana
  elementos = [] #Lista donde se va a enumerar los números correspondiente a la longitud de M
  mediana_permu = [] #Número o lista, dependiendo si es par o impar M
  median = [] #Lista donde voy a guardar la o las mediana(s)
  orden = sorted(diccio.items(), key=lambda x:x[1]) #Ordenar con respecto a los valores del diccionario

  for me in range(len(orden)): #Correr esto M veces
    elementos.append(me) #Agregar de manera ascendente número de 0 a M

  if len(elementos)%2 != 0: #Si es impar
    per_med = np.median(np.array(elementos)) #Sacar la mediana
    per_med = int(per_med) #Volverlo entero
    mediana_permu = orden[per_med][0] #Obtener la llave que está en la posicón [per_med], o sea, la mediana
    median.append(bests[mediana_permu]) #Agregar a la lista de la mediana la permutación y su costo
  else: #Si es par
    per_med = np.median(np.array(elementos)) #Sacar la mediana
    per_med_izq = per_med//1 #Hacer piso. Este número va a corresponder a la mediana en la posición izquierda
    per_med_izq = int(per_med_izq) #Convertir a entero
    per_med_der = per_med+.5 #Sumarle .5 para que corresponda a la mediana de la posición derecha
    per_med_der = int(per_med_der) #Volverlo entero
    mediana_permu.append(orden[per_med_izq][0]) #Agregar a la lista la llave que está en la posicón [per_med_izq], o sea, la mediana izquierda
    mediana_permu.append(orden[per_med_der][0]) #Hacer lo mismo pero con la mediana derecha
    median.append(bests[mediana_permu[0]]) #Agregar a la lista de la mediana la permutación izquierda y su costo.
    median.append(bests[mediana_permu[1]]) #Agregar a la lista de la mediana la permutación derecha y su costo.


  #Media
  mean_costs = [] #Lista donde voy a guardar los costos de cada permutación obtenida de las ejecuciones.

  for mc in range(len(orden)): #Correr esto M veces
    mean_costs.append(orden[mc][1]) #Agregar el costo en la posición [mc] a la lista

  mean = sum(mean_costs)/len(mean_costs) #Sacar el promedio de los costos


  #Desviación estándar
  standard_deviation = statistics.stdev(mean_costs) #Desviacíon estándar de los costos de las M permutaciones


  #Para que veas cada una de las ejecuciones, por si crees que te estoy chamaqueando :vvv
  #print(bests,'\n''Mejor solución: ', best,'\n''Peor solución: ', worst,'\n''Mediana(s): ', median,'\n''Media: ', mean, '\n''Desviación estándar: ', standard_deviation)
  

  print('Mejor solución: ', best,'\n''Peor solución: ', worst,'\n''Mediana(s): ', median,'\n''Media: ', mean, '\n''Desviación estándar: ', standard_deviation)


mejor_sol_M(M)

Mejor solución:  [([0, 1, 4, 7, 2, 6, 9, 8, 3, 5], 248.0)] 
Peor solución:  [([0, 5, 3, 8, 9, 6, 2, 7, 1, 4], 271.0)] 
Mediana(s):  [([0, 2, 7, 6, 9, 8, 3, 4, 1, 5], 269.0), ([0, 5, 3, 8, 9, 6, 2, 7, 1, 4], 271.0)] 
Media:  263.0 
Desviación estándar:  11.64474130240771
